# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [195]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [258]:
# load data from database
def load_data(db_filepath='DisasterResponse.db', sample_frac=1.0):
    """Load the data from database.
    Args:
        db_filepath: str
            Filepath to the SQL database.
        sample_frac: int
            Apply random subsampling and select only frac of entries.
    Returns:
        X, Y: np.ndarray or pandas.DataFrame
            Matrices of predictors (messages) and responses.
    """
    
    engine = create_engine('sqlite:///'+db_filepath)
    df = pd.read_sql_table('merged', engine).sample(frac=sample_frac, random_state=42)
    X = df['message']
    Y = df[['related', 'request', 'offer', 'aid_related', 'infrastructure_related', 'weather_related']]
#     Y = df[ [c for c in df.columns if 'related' in c] ]
#     Y = df[ [c for c in df.columns if c not in ('id', 'message', 'original')] ]
#     Y = Y.replace({'genre':{'direct':0, 'news':1, 'social':2}})

    category_names = Y.columns

    return X, Y, category_names

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('merged', engine).sample(frac=1.0, random_state=42)
X = df['message']
Y = df[ [c for c in df.columns if c not in ('id', 'message', 'original')] ]
df.describe()

df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [252]:
# Investigate the hierarchy of responses
df['req_or_off'] = df.request | df.offer
df['any_related'] = df.aid_related | df.infrastructure_related | df.weather_related

df1 = df.query("related == 1")

df1.query("req_or_off==0")[['req_or_off', 'direct_report']+[c for c in df.columns if 'related' in c]].describe()

df1.query("offer==1").message

11096    I have many towels ( large , as well as hand t...
11293    When and Where should hot meals be delivered t...
11335    We did a large donation drive in our community...
75             I am a driver, a mechanic ,. I want to help
23986    Following the attack on Ngouboua village by ar...
10646                    Clothes sorting . Giving out food
10854    I can bring batteries , toiletries , non-peris...
11202    Looking to donate new ( several-year shelf lif...
25301    The returnees were provided with UNHCR's stand...
18098    - UNFPA, WHO and UNICEF are building over 100 ...
10992    I 'm a nutritionist and would be also happy to...
10607    i 'm also good with food prep if there 's a ne...
21684    UNICEF has already sent supplies to affected a...
11187    Will provide & serve nonperishable food . Also...
25201    MSF teams are active with one Zodiac (inflatab...
25291    World Bank Country Director Xian Zhu told a pr...
18008    This assistance, valued at $200.7 million, was.

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """Turn a text string into an array of words.
        Args:
            text: str
        Returns:
            A list of lemmatised tokens (words).
    """
    
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    
    text = nltk.word_tokenize(text)
    
    text = [w for w in text if w not in stopwords.words('english')]
    
    # Reduce words to their root form
    text = [WordNetLemmatizer().lemmatize(w, pos='v') for w in text]
    
    return text

text = 'This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the MultiOutput Classifier helpful for predicting multiple target variables.'

print(tokenize(text))

['machine', 'pipeline', 'take', 'message', 'column', 'input', 'output', 'classification', 'result', 'categories', 'dataset', 'may', 'find', 'multioutput', 'classifier', 'helpful', 'predict', 'multiple', 'target', 'variables']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### 3.1 Classify Related/Unrelated messages

In [287]:
X, Y, cat_names = load_data(sample_frac=1.0)
X_train_txt, X_test_txt, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.25)

In [301]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
class LengthEstimator(BaseEstimator, TransformerMixin):
    """A custom transformer class to estimate the length of a message."""

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, **fit_params):
        return pd.DataFrame(pd.Series(X).str.len())
    
class DenseTransformer(TransformerMixin):
    """Transforms a sparse matrix to its dense representation."""

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()
    
pl_TextVectorise = Pipeline([
                        ('vectr', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('dense', DenseTransformer()),
                        ('scale', StandardScaler())
                ])

pl_FullNLP = FeatureUnion([
                            ('txtvec', pl_TextVectorise),
                            ('length', LengthEstimator())
                    ])

X_train = pl_FullNLP.fit_transform(X_train_txt)
X_test = pl_FullNLP.transform(X_test_txt)

In [302]:
X_test

array([[ -1.53493823e-02,  -7.13666986e-03,  -7.13666986e-03, ...,
         -7.13666986e-03,  -7.13666986e-03,   1.05000000e+02],
       [ -1.53493823e-02,  -7.13666986e-03,  -7.13666986e-03, ...,
         -7.13666986e-03,  -7.13666986e-03,   5.80000000e+01],
       [ -1.53493823e-02,  -7.13666986e-03,  -7.13666986e-03, ...,
         -7.13666986e-03,  -7.13666986e-03,   1.86000000e+02],
       ..., 
       [ -1.53493823e-02,  -7.13666986e-03,  -7.13666986e-03, ...,
         -7.13666986e-03,  -7.13666986e-03,   1.88000000e+02],
       [ -1.53493823e-02,  -7.13666986e-03,  -7.13666986e-03, ...,
         -7.13666986e-03,  -7.13666986e-03,   2.07000000e+02],
       [ -1.53493823e-02,  -7.13666986e-03,  -7.13666986e-03, ...,
         -7.13666986e-03,  -7.13666986e-03,   1.32000000e+02]])

In [ ]:
base_clsfr = LogisticRegression(solver='lbfgs', random_state=0)
pipeline = Pipeline([
#                 ('vectr', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer()),
# #                 ('clf', RandomForestClassifier()),
# #                  ('dense', DenseTransformer()),
# #                  ('scale', StandardScaler()),
#                 ('clsfr', LinearSVC(C = 1, dual=False))
                ('clsfr', MultiOutputClassifier(SVC(gamma='auto')))
#                 ('chain', ClassifierChain(base_clsfr, order = [0,1,2,3,4,5]))
           ])



# train classifier
pipeline.fit(X_train, Y_train)

# predict on test data
Y_pred = pipeline.predict(X_test)

Y_pred = pd.DataFrame(Y_pred, columns = Y_test.columns)
Y_pred.describe()

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [304]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84      4966
           1       0.47      0.56      0.51      1085
           2       0.00      0.00      0.00        37
           3       0.59      0.64      0.62      2710
           4       0.11      0.12      0.12       416
           5       0.61      0.65      0.63      1820

   micro avg       0.67      0.70      0.68     11034
   macro avg       0.44      0.47      0.45     11034
weighted avg       0.68      0.70      0.69     11034
 samples avg       0.55      0.54      0.51     11034



C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [197]:
#
pipeline = Pipeline([
                ('vectr', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
#                 ('clf', RandomForestClassifier()),
                #         ('dense', DenseTransformer()),
#                 ('scale', StandardScaler()),
#                 ('clsfr', LinearSVC())
                ('clsfr', MultiOutputClassifier(LinearSVC()))
           ])
# pipeline.get_params()

In [281]:
pipeline.get_params()
# print(classification_report(Y_test['related'], Y_pred))
type(model)

sklearn.pipeline.Pipeline

In [242]:
parameters = {'vectr__max_df' : [1.0, 0.8, 0.7, 0.6],
              'vectr__min_df' : [0.0, 0.01]}
parameters = {'clsfr__C' : [0.1, 0.4, 0.5, 0.6, 1.0, 2.0, 5.0, 10.0, 100.0],
              'clsfr__penalty': ['l1', 'l2']}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=4)

cv.fit(X_train, Y_train['related'])

model = cv.best_estimator_

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] clsfr__C=0.1, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.1, clsfr__penalty=l1, score=0.7690355329949239, total=   0.0s
[CV] clsfr__C=0.1, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.1, clsfr__penalty=l1, score=0.7735368956743003, total=   0.0s
[CV] clsfr__C=0.1, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.1, clsfr__penalty=l1, score=0.7704081632653061, total=   0.0s
[CV] clsfr__C=0.1, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.1, clsfr__penalty=l1, score=0.7678571428571429, total=   0.0s
[CV] clsfr__C=0.1, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.1, clsfr__penalty=l1, score=0.7627551020408163, total=   0.0s
[CV] clsfr__C=0.1, clsfr__penalty=l2 .................................
[CV]  clsfr__C=0.1, clsfr__penalty=l2, score=0.7690355329949239, total=   0.0s
[CV] clsfr__C=0.1, clsfr__penalty=l2 ..

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s


[CV]  clsfr__C=0.5, clsfr__penalty=l2, score=0.811704834605598, total=   0.0s
[CV] clsfr__C=0.5, clsfr__penalty=l2 .................................
[CV]  clsfr__C=0.5, clsfr__penalty=l2, score=0.7933673469387755, total=   0.0s
[CV] clsfr__C=0.5, clsfr__penalty=l2 .................................
[CV]  clsfr__C=0.5, clsfr__penalty=l2, score=0.8010204081632653, total=   0.0s
[CV] clsfr__C=0.5, clsfr__penalty=l2 .................................
[CV]  clsfr__C=0.5, clsfr__penalty=l2, score=0.8010204081632653, total=   0.0s
[CV] clsfr__C=0.6, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.6, clsfr__penalty=l1, score=0.7715736040609137, total=   0.0s
[CV] clsfr__C=0.6, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.6, clsfr__penalty=l1, score=0.7964376590330788, total=   0.0s
[CV] clsfr__C=0.6, clsfr__penalty=l1 .................................
[CV]  clsfr__C=0.6, clsfr__penalty=l1, score=0.7806122448979592, total=   0.0s
[CV] clsfr__C=0.6, cls

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV]  clsfr__C=10.0, clsfr__penalty=l1, score=0.7506361323155216, total=   0.0s
[CV] clsfr__C=10.0, clsfr__penalty=l1 ................................
[CV]  clsfr__C=10.0, clsfr__penalty=l1, score=0.7831632653061225, total=   0.0s
[CV] clsfr__C=10.0, clsfr__penalty=l1 ................................
[CV]  clsfr__C=10.0, clsfr__penalty=l1, score=0.7653061224489796, total=   0.0s
[CV] clsfr__C=10.0, clsfr__penalty=l1 ................................
[CV]  clsfr__C=10.0, clsfr__penalty=l1, score=0.7576530612244898, total=   0.0s
[CV] clsfr__C=10.0, clsfr__penalty=l2 ................................
[CV]  clsfr__C=10.0, clsfr__penalty=l2, score=0.7690355329949239, total=   0.0s
[CV] clsfr__C=10.0, clsfr__penalty=l2 ................................
[CV]  clsfr__C=10.0, clsfr__penalty=l2, score=0.7837150127226463, total=   0.0s
[CV] clsfr__C=10.0, clsfr__penalty=l2 ................................
[CV]  clsfr__C=10.0, clsfr__penalty=l2, score=0.7780612244897959, total=   0.0s
[CV] clsfr__C=

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    3.1s finished


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [151]:
X, Y, cat_names = load_data(sample_frac=0.1)
# Y = Y.iloc[:, 0]
# print(Y.value_counts())
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.25)

# train classifier
pipeline.fit(X_train, Y_train)

# print(Y.shape)

# predict on test data
Y_pred = pipeline.predict(X_test)

# # display results
# display_results(Y_test, Y_pred)

In [78]:
# train only the word processor
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# Building the classification part of the pipeline
clsfr = MultiOutputClassifier(LinearSVC())

clsfr.fit(X_train, Y_train)
Y_pred = clsfr.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [201]:
for i in range(4):
    print(classification_report(Y_test.values[:,i], Y_pred[:,i]))

IndexError: too many indices for array

### 6. Improve your model
Use grid search to find better parameters. 

In [147]:
parameters = {'vectr__max_df' : [1.0, 0.8, 0.7, 0.6],
              'vectr__min_df' : [0.0, 0.01]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_weighted', cv=3, verbose=4)

cv.fit(X_train, Y_train)

model = cv.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] vectr__max_df=1.0, vectr__min_df=0.0 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  vectr__max_df=1.0, vectr__min_df=0.0, score=0.7127898967590667, total=  16.4s
[CV] vectr__max_df=1.0, vectr__min_df=0.0 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.8s remaining:    0.0s


[CV]  vectr__max_df=1.0, vectr__min_df=0.0, score=0.7212080304774412, total=  16.9s
[CV] vectr__max_df=1.0, vectr__min_df=0.0 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   55.9s remaining:    0.0s


[CV]  vectr__max_df=1.0, vectr__min_df=0.0, score=0.7413184335905869, total=  16.8s
[CV] vectr__max_df=1.0, vectr__min_df=0.01 ...........................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.4min remaining:    0.0s


[CV]  vectr__max_df=1.0, vectr__min_df=0.01, score=0.7133483865761051, total=  16.8s
[CV] vectr__max_df=1.0, vectr__min_df=0.01 ...........................
[CV]  vectr__max_df=1.0, vectr__min_df=0.01, score=0.7265031659935167, total=  16.9s
[CV] vectr__max_df=1.0, vectr__min_df=0.01 ...........................
[CV]  vectr__max_df=1.0, vectr__min_df=0.01, score=0.7215772256285989, total=  16.9s
[CV] vectr__max_df=0.8, vectr__min_df=0.0 ............................
[CV]  vectr__max_df=0.8, vectr__min_df=0.0, score=0.7127898967590667, total=  17.3s
[CV] vectr__max_df=0.8, vectr__min_df=0.0 ............................
[CV]  vectr__max_df=0.8, vectr__min_df=0.0, score=0.7212080304774412, total=  17.8s
[CV] vectr__max_df=0.8, vectr__min_df=0.0 ............................
[CV]  vectr__max_df=0.8, vectr__min_df=0.0, score=0.7413184335905869, total=  17.2s
[CV] vectr__max_df=0.8, vectr__min_df=0.01 ...........................
[CV]  vectr__max_df=0.8, vectr__min_df=0.01, score=0.71334838657610

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 11.5min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [148]:
Y_pred = model.predict(X_test)
model.get_params()

{'clsfr': MultiOutputClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
            n_jobs=None),
 'clsfr__estimator': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'clsfr__estimator__C': 1.0,
 'clsfr__estimator__class_weight': None,
 'clsfr__estimator__dual': True,
 'clsfr__estimator__fit_intercept': True,
 'clsfr__estimator__intercept_scaling': 1,
 'clsfr__estimator__loss': 'squared_hinge',
 'clsfr__estimator__max_iter': 1000,
 'clsfr__estimator__multi_class': 'ovr',
 'clsfr__estimator__penalty': 'l2',
 'clsfr__estimator__random_state': None,
 'clsfr__estimator__tol': 0.0001,
 'clsfr__estimator__verbose': 0,
 'clsfr__n_jobs': N

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.